In [1]:
import numpy as np

def generate_synthetic_data(low_variance=False, n_samples=10000, sample_size=1000):
    # Define a known precision matrix (inverse covariance) with a known sparsity pattern
    if low_variance:
        true_precision = np.array([
            [1.0, -0.5, 0.0, 0.0, 0.0],
            [-0.5, 1.0, -0.4, 0.0, 0.0],
            [0.0, -0.4, 1.0, -0.3, 0.0],
            [0.0, 0.0, -0.3, 1.0, -0.2],
            [0.0, 0.0, 0.0, -0.2, 1.0]
        ])
    else:
        true_precision = np.array([
            [10.0, -1.8, -1.5, -1.2, -0.9],
            [-1.8, 10.0, -1.6, -1.3, -1.0],
            [-1.5, -1.6, 10.0, -1.4, -1.1],
            [-1.2, -1.3, -1.4, 10.0, -1.2],
            [-0.9, -1.0, -1.1, -1.2, 10.0]
        ])
    
    # Ensure the matrix is symmetric and positive definite
    true_precision = (true_precision + true_precision.T) / 2
    eigvals = np.linalg.eigvals(true_precision)
    if np.any(eigvals <= 0):
        raise ValueError("Precision matrix is not positive definite")
    
    # Compute the true covariance matrix as the inverse of the precision matrix
    true_covariance = np.linalg.inv(true_precision)
    
    # Generate data from a multivariate normal distribution
    data = np.random.multivariate_normal(mean=np.zeros(5), cov=true_covariance, size=n_samples)
    
    # Reshape data into segments
    num_full_samples = n_samples // sample_size
    sampled_data = data[:num_full_samples * sample_size]
    samples = sampled_data.reshape((num_full_samples, sample_size, 5))
    
    # Calculate the empirical covariance matrices for each segment
    covariances = np.array([np.cov(sample, rowvar=False) for sample in samples])
    
    return samples, covariances, true_precision, true_covariance

# Usage
samples, covariances, true_precision, true_covariance = generate_synthetic_data()
print("Sample shape:", samples.shape)  # Should be (num_full_samples, sample_size, num_variables)
print("Covariance matrices shape:", covariances.shape)  # Should be (num_full_samples, num_variables, num_variables)
print("True precision matrix:\n", true_precision)
print("True covariance matrix:\n", true_covariance)


Sample shape: (10, 1000, 5)
Covariance matrices shape: (10, 5, 5)
True precision matrix:
 [[10.  -1.8 -1.5 -1.2 -0.9]
 [-1.8 10.  -1.6 -1.3 -1. ]
 [-1.5 -1.6 10.  -1.4 -1.1]
 [-1.2 -1.3 -1.4 10.  -1.2]
 [-0.9 -1.  -1.1 -1.2 10. ]]
True covariance matrix:
 [[0.11403508 0.02988324 0.02730285 0.02368304 0.01909676]
 [0.02988324 0.1152567  0.02866834 0.02502757 0.02037199]
 [0.02730285 0.02866834 0.11457048 0.02556241 0.02099433]
 [0.02368304 0.02502757 0.02556241 0.11218326 0.02090809]
 [0.01909676 0.02037199 0.02099433 0.02090809 0.10857425]]


In [2]:
def soft_threshold_odd(a, lambd, rho):

    # The off-diagonal Lasso penalty function.
    # Computes the Z0 update with off-diagonal
    # soft-threshold operator

    parameter = lambd/rho
    dimension = np.shape(a)[0]
    e = np.eye(dimension)
    for i in range(dimension - 1):
        for j in range(i + 1, dimension):
            if abs(a[i, j]) > parameter:
                result = np.sign(a[i, j])*(
                    abs(a[i, j]) - parameter)
                e[i, j] = result
                e[j, i] = result
    return e


def element_wise(a, beta, rho):

    # The element-wise l1 penalty function.
    # Used in (Z1, Z2) update

    eta = 2*beta/rho
    dimension = np.shape(a)[0]
    e = np.zeros((dimension, dimension))
    for i in range(dimension):
        for j in range(dimension):
            if abs(a[i, j]) > eta:
                e[i, j] = np.sign(a[i, j])*(
                    abs(a[i, j]) - eta)
    return e


def group_lasso(a, beta, rho):

    # The Group Lasso l2 penalty function.
    # Used in (Z1, Z2) update

    eta = 2*beta/rho
    dimension = np.shape(a)[0]
    e = np.zeros((dimension, dimension))
    for j in range(dimension):
        l2_norm = np.linalg.norm(a[:, j])
        if l2_norm > eta:
            e[:, j] = (1 - eta/l2_norm)*a[:, j]
    return e


def perturbed_node(theta_pre, theta, u1, u2, beta, rho, ct, cc):

    # The row-column overlap penalty function (Perturbed Node).
    # Used in (Z1, Z2 update)
    # Computes the update as a sub-ADMM algorithm,
    # as no closed form update exists.

    """ Initialize ADMM algorithm """
    dimension = np.shape(theta)[0]
    y1 = np.ones((dimension, dimension))
    y2 = np.ones((dimension, dimension))
    v = np.ones((dimension, dimension))
    w = np.ones((dimension, dimension))
    uu1 = np.zeros((dimension, dimension))
    uu2 = np.zeros((dimension, dimension))

    """ Run algorithm """
    iteration = 0
    y_pre = []
    stopping_criteria = False
    while iteration < 1000 and stopping_criteria is False:
        a = (y1 - y2 - w - uu1 + (w.transpose() - uu2).transpose())/2

        """ V Update """
        v = group_lasso(a, beta, rho)

        """ W, Y1, Y2 Update """
        b = np.zeros((3*dimension, dimension))
        b[0:dimension, :] = (v + uu2).transpose()
        b[dimension:2*dimension, :] = theta_pre + u1
        b[2*dimension:3*dimension, :] = theta + u2
        d = v + uu1
        wyy = np.dot(cc, 2*b - np.dot(ct, d))
        w = wyy[0:dimension, :]
        y1 = wyy[dimension:2*dimension, :]
        y2 = wyy[2*dimension:3*dimension, :]

        """ UU1, UU2 Update """
        uu1 = uu1 + (v + w) - (y1 - y2)
        uu2 = uu2 + v - w.transpose()

        """ Check stopping criteria """
        if iteration > 0:
            dif = y1 - y_pre
            fro_norm = np.linalg.norm(dif)
            if fro_norm < 1e-3:
                stopping_criteria = True
        y_pre = list(y1)
        iteration += 1
    return (y1, y2)

In [39]:

def theta_update(z0s, z1s, z2s, u0s, u1s, u2s, emp_cov_mat, eta, dimension, blocks):
    thetas = []
    for i in range(blocks):
        #print(i)
        a = (z0s[i] + z1s[i] + z2s[i] - u0s[i] - u1s[i] - u2s[i]) / 3
        at = a.transpose()
        m = eta * (a + at) / 2 - emp_cov_mat[i]
        d, q = np.linalg.eig(m)
        qt = q.transpose()
        sqrt_matrix = np.sqrt(d**2 + 4 / eta * np.ones(dimension))
        diagonal = np.diag(d) + np.diag(sqrt_matrix)
        theta = np.real(eta / 2 * np.dot(np.dot(q, diagonal), qt))
        thetas.append(theta)
    return thetas

def z0_update(thetas, u0s, lambd, rho, blocks, soft_threshold_odd_func):
    return [soft_threshold_odd_func(thetas[i] + u0s[i], lambd, rho) for i in range(blocks)]

def z1_z2_update(thetas, u1s, u2s, beta, rho, blocks, penalty_function):

    dimension = thetas[0].shape[0]
    z1s = [np.eye(dimension) for _ in range(blocks)]
    z2s = [np.eye(dimension) for _ in range(blocks)]
    
    if penalty_function == "perturbed_node":
        for i in range(1, blocks):
            z1s[i - 1], z2s[i] = perturbed_node(thetas[i - 1], thetas[i], u1s[i - 1], u2s[i], beta, rho)
    else:
        aa = [thetas[i] - thetas[i - 1] + u2s[i] - u1s[i - 1] for i in range(1, blocks)]

        if penalty_function == "L1":
            ee = [element_wise(a, beta, rho) for a in aa]    
        if penalty_function == "L2":
            ee = [group_lasso(a, beta, rho) for a in aa]

        for i in range(1, blocks):
            summ = thetas[i - 1] + thetas[i] + u1s[i - 1] + u2s[i]
            z1s[i - 1] = 0.5 * (summ - ee[i - 1])
            z2s[i] = 0.5 * (summ + ee[i - 1])
    return z1s, z2s

def u_update(u0s, u1s, u2s, thetas, z0s, z1s, z2s, blocks):
    for i in range(blocks):
        u0s[i] += thetas[i] - z0s[i]
    for i in range(1, blocks):
        u2s[i] += thetas[i] - z2s[i]
        u1s[i - 1] += thetas[i - 1] - z1s[i - 1]
    return u0s, u1s, u2s



def check_for_none(thetas, z0s, z1s, z2s, u0s, u1s, u2s):
    for name, array in zip(
        ["thetas", "z0s", "z1s", "z2s", "u0s", "u1s", "u2s"],
        [thetas, z0s, z1s, z2s, u0s, u1s, u2s]
    ):
        for i, element in enumerate(array):
            if element is None:
                print(f"None found in {name} at index {i}")
            else:
                print(f"{name}[{i}] is not None, shape: {element.shape}")




In [41]:


def calculate_primal_residual(thetas, z0s, z1s, z2s, blocks):
    # Calculate the primal residuals for each block
    primal_residuals = []
    for i in range(blocks):
        r_0 = thetas[i] - z0s[i]
        primal_residuals.append(np.linalg.norm(r_0, 'fro'))
        
    for i in range(1, blocks):
        r_1 = thetas[i] - z1s[i - 1]
        r_2 = thetas[i] - z2s[i]
        primal_residuals.append(np.linalg.norm(r_1, 'fro'))
        primal_residuals.append(np.linalg.norm(r_2, 'fro'))
    
    # Combine residuals to get the total primal residual
    return np.sqrt(np.sum(np.square(primal_residuals)))

def calculate_dual_residual(z0s, z1s, z2s, z0s_prev, z1s_prev, z2s_prev, rho, blocks):
    # Calculate the dual residuals for each block
    dual_residuals = []
    for i in range(blocks):
        s_0 = rho * (z0s[i] - z0s_prev[i])
        dual_residuals.append(np.linalg.norm(s_0, 'fro'))
    
    for i in range(1, blocks):
        s_1 = rho * (z1s[i - 1] - z1s_prev[i - 1])
        s_2 = rho * (z2s[i] - z2s_prev[i])
        dual_residuals.append(np.linalg.norm(s_1, 'fro'))
        dual_residuals.append(np.linalg.norm(s_2, 'fro'))
    
    # Combine residuals to get the total dual residual
    return np.sqrt(np.sum(np.square(dual_residuals)))

def check_convergence(primal_residual, dual_residual, thetas, z0s, abs_tol, rel_tol):
    # Calculate primal and dual tolerances
    dimension = np.shape(thetas[0])[0]
    epsilon_primal = np.sqrt(dimension) * abs_tol + rel_tol * max(
        np.linalg.norm(np.concatenate(thetas), 'fro'), 
        np.linalg.norm(np.concatenate(z0s), 'fro')
    )
    epsilon_dual = np.sqrt(dimension) * abs_tol + rel_tol * np.linalg.norm(rho * np.concatenate(thetas), 'fro')

    # Check if both residuals are within their respective tolerances
    return primal_residual <= epsilon_primal and dual_residual <= epsilon_dual

# Example ADMM loop
def admm_loop(z0s, z1s, z2s, u0s, u1s, u2s, emp_cov_mat, eta, lambd, beta, rho, abs_tol, rel_tol, dimension, blocks, max_iters=10000):
    thetas = [np.zeros((dimension, dimension)) for _ in range(blocks)]
    
    for k in range(max_iters):
        # Store previous Z values for dual residual calculation
        z0s_prev, z1s_prev, z2s_prev = z0s.copy(), z1s.copy(), z2s.copy()
        
        # Update theta, z, and u in each ADMM iteration
        thetas = theta_update(z0s, z1s, z2s, u0s, u1s, u2s, emp_cov_mat, eta, dimension, blocks)
        z0s = z0_update(thetas, u0s, lambd, rho, blocks, soft_threshold_odd)
        z1s, z2s = z1_z2_update(thetas, u1s, u2s, beta, rho, blocks, "L1")
        u0s, u1s, u2s = u_update(u0s, u1s, u2s, thetas, z0s, z1s, z2s, blocks)
        
        # Calculate primal and dual residuals
        primal_residual = calculate_primal_residual(thetas, z0s, z1s, z2s, blocks)
        dual_residual = calculate_dual_residual(z0s, z1s, z2s, z0s_prev, z1s_prev, z2s_prev, rho, blocks)


        # Example usage after ADMM loop or at any checkpoint
        #check_for_none(thetas, z0s, z1s, z2s, u0s, u1s, u2s)
        
        # Check convergence
        if check_convergence(primal_residual, dual_residual, thetas, z0s, abs_tol, rel_tol):
            print(f"Converged in {k+1} iterations")
            break
    else:
        print("Max iterations reached without convergence")

    return thetas, z0s, z1s, z2s, u0s, u1s, u2s


In [42]:
# Set parameters
eta = 0.1           # Parameter for theta update
lambd = 0.5         # Parameter for Lasso penalty in z0 update
beta = 0.5          # Parameter for overlap penalty
rho = 1.0           # ADMM penalty parameter
abs_tol = 1e-4      # Absolute tolerance for convergence
rel_tol = 1e-3      # Relative tolerance for convergence
dimension = true_precision.shape[0]
blocks = covariances.shape[0]  # Number of blocks (segments)

# Initialize variables
z0s = [np.eye(dimension) for _ in range(blocks)]
z1s = [np.eye(dimension) for _ in range(blocks)]
z2s = [np.eye(dimension) for _ in range(blocks)]
u0s = [np.zeros((dimension, dimension)) for _ in range(blocks)]
u1s = [np.zeros((dimension, dimension)) for _ in range(blocks)]
u2s = [np.zeros((dimension, dimension)) for _ in range(blocks)]

# Run ADMM loop
thetas, z0s, z1s, z2s, u0s, u1s, u2s = admm_loop(
    z0s, z1s, z2s, u0s, u1s, u2s, covariances, eta, lambd, beta, rho,
    abs_tol, rel_tol, dimension, blocks
)

# Print the result
print("Estimated precision matrices (Thetas):")
for i, theta in enumerate(thetas):
    print(f"Theta {i}:\n", theta)
print("\nTrue precision matrix:\n", true_precision)

Converged in 2232 iterations
Estimated precision matrices (Thetas):
Theta 0:
 [[ 9.98963945e-01 -1.13204451e-03 -7.69228943e-04 -6.22892166e-04
  -3.22267559e-04]
 [-1.13204451e-03  9.98963945e-01 -8.60509966e-04 -7.12148039e-04
  -2.02550328e-04]
 [-7.69228943e-04 -8.60509966e-04  9.98964280e-01 -6.01777896e-04
  -2.94898657e-04]
 [-6.22892166e-04 -7.12148039e-04 -6.01777896e-04  9.98964397e-01
  -2.95854268e-04]
 [-3.22267559e-04 -2.02550328e-04 -2.94898657e-04 -2.95854268e-04
   9.98964532e-01]]
Theta 1:
 [[ 9.98968253e-01 -1.09063100e-03 -7.68554782e-04 -6.11542473e-04
  -2.76874102e-04]
 [-1.09063100e-03  9.98968222e-01 -8.34833141e-04 -6.98808898e-04
  -2.27718808e-04]
 [-7.68554782e-04 -8.34833141e-04  9.98968586e-01 -5.78318815e-04
  -2.88519399e-04]
 [-6.11542473e-04 -6.98808898e-04 -5.78318815e-04  9.98968593e-01
  -2.92014038e-04]
 [-2.76874102e-04 -2.27718808e-04 -2.88519399e-04 -2.92014038e-04
   9.98968761e-01]]
Theta 2:
 [[ 9.98970852e-01 -1.03560168e-03 -7.71100257e-04 

: 

In [26]:
u2s

[array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 array([[ 9.51558008e-05,  1.07044695e-04,  1.19297290e-04,
          8.32633290e-05,  1.94268489e-04],
        [ 1.07044695e-04,  7.80002160e-05, -7.56150971e-06,
          3.50031204e-05,  1.08673775e-04],
        [ 1.19297290e-04, -7.56150971e-06,  2.75235574e-04,
          3.00896462e-04,  1.55215122e-04],
        [ 8.32633290e-05,  3.50031204e-05,  3.00896462e-04,
         -5.01288548e-05, -2.33271334e-05],
        [ 1.94268489e-04,  1.08673775e-04,  1.55215122e-04,
         -2.33271334e-05,  4.33256325e-04]]),
 array([[-1.14614705e-04,  2.21847700e-04, -1.47445642e-04,
          3.04134719e-05, -1.60165189e-04],
        [ 2.21847700e-04, -3.24207814e-05, -1.07231466e-05,
          5.33973216e-05, -1.24200458e-04],
        [-1.47445642e-04, -1.07231466e-05, -1.87725177e-04,
         -1.20023575e-04, -2.43332006e-04],
        [ 3.04